# Основы веб-скрапинга и работы с API

**Задание 1**
**Обязательная часть**
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

In [28]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [43]:
def get_links(url, query):
    links_list = []
    params = {
        'q': query,
        'order_by': 'relevance'
    }
    res = requests.get(URL, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    search_blocks = soup.find_all('h2', class_='post__title')
    links_list = list(map(lambda x: x.find('a').get('href'), search_blocks))
    return links_list

KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'
all_links = []
for word in KEYWORDS:
    links = get_links(URL, word)
    all_links = all_links + links

In [48]:
habr_search = pd.DataFrame()
for links in all_links:
    soup = BeautifulSoup(requests.get(links).text, 'html.parser')
    time.sleep(0.3)
    date = soup.find('span', class_='post__time').text
    title = soup.find('span', class_='post__title-text').text
    article_text = soup.find('div', class_='post__text post__text-html post__text_v1').text
    row = {'date': date, 'title': title, 'link': links, 'article_text': article_text}
    habr_search = pd.concat([habr_search, pd.DataFrame([row])]) 
habr_search.head(10)

,date,title,link,article_text
0,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"\r\n20 апреля 2020 года, спустя почти десять л..."
0,3 марта 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P..."
0,3 июня 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/,\n\r\n31 мая 2020 года разработчик Джош Мур (J...
0,31 октября 2019 в 10:02,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/t/473926/,\n\r\nСоздатель языка программирования Python ...
0,16 сентября 2020 в 16:48,Астрономам порекомендовали меньше использовать...,https://habr.com/ru/news/t/519414/,\n\r\nАстрономы из Лейденской обсерватории опу...
0,9 июля 2012 в 10:21,Мысли о Python 3,https://habr.com/ru/post/147281/,Предлагаю вашему вниманю пересказ замечательно...
0,9 декабря 2012 в 07:16,Дружим Python 3 с MS Visual C++. Строим мост в...,https://habr.com/ru/post/161931/,Всем доброго {daytime}!\n\r\nСегодня пришла по...
0,3 февраля 2013 в 17:26,Объединяя C++ и Python. Тонкости Boost.Python....,https://habr.com/ru/post/168083/,Boost.Python во всех отношениях замечательная ...
0,10 февраля 2013 в 12:24,Конвертация типов в Boost.Python. Делаем преоб...,https://habr.com/ru/post/168827/,Данная статья не является продолжением повеств...
0,17 февраля 2013 в 17:37,Путешествие исключений между C++ и Python или ...,https://habr.com/ru/post/169639/,В этой главе сказа про дружбу C++ и Python буд...


**Задание 2**
**Обязательная часть**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [35]:
EMAIL = ['xxx@x.ru, yyy@y.com']

In [41]:
def check(EMAIL: list = None):
    if EMAIL is None:
        EMAIL = ['xxx@x.ru', 'yyy@y.com']
    df = pd.DataFrame()
    for mail in EMAIL:
        req = requests.post('https://digibody.avast.com/v1/web/leaks/',
                            json={'email': mail})
        dat = pd.json_normalize(req.json()['value'])[[
                                                'leak_date',
                                                'leak_info.title',
                                                'leak_info.description',
                                               ]]
        dat.insert(0, 'mail', mail)
        df = df.append(dat)
        time.sleep(0.33)
    df.leak_date = pd.to_datetime(df.leak_date, unit='ms', origin='unix')
    df.columns=['почта', 'дата утечки', 'источник утечки', 'описание утечки']
    return df

check().tail()

,почта,дата утечки,источник утечки,описание утечки
20,yyy@y.com,2020-01-03,AZCentral,"At an unconfirmed date, online Arizona newspap..."
21,yyy@y.com,2018-02-18,Netlog,Netlog (formerly known as Facebox and Bingbox)...
22,yyy@y.com,2020-06-25,"""Cash Cloud"" Retail Combo List","At an unconfirmed date, this ""Cash Cloud"" Reta..."
23,yyy@y.com,2020-02-13,Sensitive Source,This source has been marked as sensitive due t...
24,yyy@y.com,2016-10-23,iMesh,"In June 2016, a cache of over 51 million user ..."
